In [15]:
import gymnasium as gym
from gymnasium.utils import play
import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from tensorflow.keras.models import load_model
import keras
import numpy as np

from keras import backend 

In [ ]:
env = gym.make('ALE/Boxing-v5', render_mode = 'human', obs_type = 'grayscale')
#or for grayscale
#env = gym.make

In [20]:
model = load_model('reinforcement-learning/models/modelOnline100.h5')


TypeError: Error when deserializing class 'Conv2D' using config={'name': 'conv2d', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': [None, 150, 100, 5], 'filters': 32, 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}.

Exception encountered: module 'keras.src.backend' has no attribute 'RandomGenerator'

In [24]:
env.reset()
for i in range(140):
    env.render()
    a = env.step(env.action_space.sample())
    env.render()
print(a[1:])

c:\Users\leomo\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


(0.0, False, False, {'lives': 0, 'episode_frame_number': 560, 'frame_number': 560})


In [3]:
def crop_image(image):
    return image[30:180, 30:130]

def first_stack(image):
    return np.stack((image, image, image, image,image), axis=2)

def stack_image(image, stack):
    #inputs (150,100) and (1, 150, 100, 5)
    return np.append(stack[:, :, :, 1:], np.expand_dims(image, 2), axis=3)

def reshape_image(image):
    return np.expand_dims(image, 0)

In [7]:
# Create a model with keras that takes in the image and the np.array of the reward of all possible actions
# and outputs the expected reward of each action, using a convolutional neural network and mse loss
batch_size = 1

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 100, 5)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(18, activation='linear'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2), loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 98, 32)       1472      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 47, 32)        9248      
                                                                 
 flatten (Flatten)           (None, 108288)            0         
                                                                 
 dense (Dense)               (None, 64)                6930496   
                                                                 
 dense_1 (Dense)             (None, 18)                1170      
                                                        

In [6]:
def gen_labels(env):
    labels = np.zeros(18)
    for k in range(18):
        old_state = env.unwrapped.clone_state(include_rng=False)
        labels[i] = env.step(k)[1]
        env.unwrapped.restore_state(old_state)
    return tf.stack(labels.reshape((1, 18)))

In [7]:
# Now we need to train the model
# We will generate the vector of labels with the environment.step() function
# We will fit the model using those labels and the images
# We will use the model to predict the next action and then use that action to generate the next image and label
# We will repeat this process until the model is trained

# We will use the epsilon-greedy algorithm to generate the labels

epochs = 3
epsilon = 0.1

cropped_image = crop_image(env.reset()[0])
img = tf.stack(first_stack(cropped_image).reshape((1, 150, 100, 5)))

for i in range(epochs):
    cropped_image = crop_image(env.reset()[0])
    img = tf.stack(first_stack(cropped_image).reshape((1, 150, 100, 5)))
    for j in range(1000):
        # Generate the labels
        labels = gen_labels(env)

        # Fit the model
        model.fit(img,labels, epochs=1, batch_size=batch_size)

        # Predict the next action
        out = model.predict(img)
        action = np.argmax(out)

        # Epsilon-greedy
        if np.random.rand() < epsilon:
            action = env.action_space.sample()

        # Generate the next image and label
        cropped_image = crop_image(env.step(action)[0])
        print(cropped_image.shape, img.shape)
        img = stack_image(cropped_image, img)
        img = tf.stack(img.reshape((1, 150, 100, 5)))

        # Check if the game is over
        if env.unwrapped.ale.game_over():
            break
    print('Epoch', i, 'finished')
    env.reset()



ValueError: cannot reshape array of size 225000 into shape (1,150,100,5)

In [59]:
a = env.reset()
img = crop_image(a[0])
img = first_stack(img)
for i in range(1000):
    env.render()
    img = crop_image(a[0])
    stack = stack_image(img, stack)
    a = env.step(np.argmax(model.predict(tf.stack(stack.reshape((1, 150, 100, 5))))[0]))
    env.render()


1/1 [==============================] - 0s 27ms/step


In [55]:
env.reset()
for i in range(140):
    env.render()
    img = tf.stack(a[0].reshape((1, 150, 100)))
    a = env.step(np.argmax(model.predict(img)))
    env.render()


ValueError: cannot reshape array of size 33600 into shape (1,150,100)

In [25]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 100, 5)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(18, activation='linear'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2), loss='mse')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 98, 32)       1472      
                                                                 
 batch_normalization_2 (Batc  (None, 148, 98, 32)      128       
 hNormalization)                                                 
                                                                 
 conv2d_5 (Conv2D)           (None, 146, 96, 32)       9248      
                                                                 
 flatten_2 (Flatten)         (None, 448512)            0         
                                                                 
 dense_4 (Dense)             (None, 64)                28704832  
                                                                 
 dense_5 (Dense)             (None, 18)                1170      
                                                      

In [26]:
batch_size = 64
model.fit(stacked_images,labels[:-4], epochs=10, batch_size=batch_size)

Epoch 1/10
197/197 [==============================] - 229s 1s/step - loss: 11977.9355
Epoch 2/10
197/197 [==============================] - 231s 1s/step - loss: 1234.1859
Epoch 3/10
197/197 [==============================] - 237s 1s/step - loss: 1216.1942
Epoch 4/10
197/197 [==============================] - 249s 1s/step - loss: 1195.2865
Epoch 5/10
197/197 [==============================] - 233s 1s/step - loss: 1172.4376
Epoch 6/10
197/197 [==============================] - 283s 1s/step - loss: 1148.2631
Epoch 7/10
197/197 [==============================] - 228s 1s/step - loss: 1123.1902
Epoch 8/10
197/197 [==============================] - 230s 1s/step - loss: 1097.5261
Epoch 9/10
197/197 [==============================] - 225s 1s/step - loss: 1071.4972
Epoch 10/10
197/197 [==============================] - 229s 1s/step - loss: 1045.2751


In [19]:
# for i in range(images.shape[0] - 4):
#     stacked_images[i] = np.stack((images[i], images[i+1], images[i+2], images[i+3], images[i+4]), axis=2)

In [4]:
# read the training data from the csv file

import pandas as pd
df = pd.read_csv('training_data.csv')

In [5]:
# append one last 0 to the end of the data
data = df.to_numpy()
data = np.append(data, data[-1])

In [6]:
# pd df to numpy array
#data = df.to_numpy()
# break the data in 15018 entries blocks

data = data.reshape((int(data.shape[0]/15018), 15018))

In [7]:
# get the last 18 entries of each block and put them in a new array
labels = data[:, -18:]

images = data[:, :-18]

#reshape images to 150, 100 and stack them
images = images.reshape((images.shape[0], 150, 100))
images.shape

#stacked_images = np.zeros((images.shape[0] - 4, 150, 100, 5))


(12610, 150, 100)

In [8]:
def generator(bs):
    while True:
        image = np.zeros((bs, 150, 100, 5))
        label = np.zeros((bs, 18))
        for b in range(bs):
            rng = np.random.randint(0, images.shape[0] - 4)
            image[b] = np.stack((images[rng], images[rng+1], images[rng+2], images[rng+3], images[rng+4]), axis=2)
            label[b] = labels[rng]
        yield image, label

In [29]:
generator().

AttributeError: 'generator' object has no attribute 'shape'

In [9]:
def stack_image(image, stack):
    #inputs (150,100, 5) and (150, 100)
    return np.append(stack[:, :, 1:], np.expand_dims(image, 2), axis=2)

TypeError: Could not locate function 'mean_squared_error'. Make sure custom classes are decorated with `@keras.utils.register_keras_serializable`. Full object config: {'module': 'builtins', 'class_name': 'function', 'config': 'mean_squared_error', 'registered_name': 'function'}

In [38]:
count = 0
while True:
    batch_size = 64
    model.fit(x = generator(batch_size), epochs=1, steps_per_epoch=64)

41/64 [==================>...........] - ETA: 35s - loss: 1020.8570

KeyboardInterrupt: 

In [11]:
image = crop_image(env.reset()[0])
fs = first_stack(image)
stack = fs
for i in range(1000):
    env.render()
    action = np.argmax(model.predict(tf.stack(stack.reshape((1, 150, 100, 5))))[0])
    print(action)
    image = crop_image(env.step(action)[0])
    stack = stack_image(image, stack)
    env.render()

c:\Users\leomo\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


1/1 [==============================] - 1s 695ms/step
17
1/1 [==============================] - 0s 61ms/step
17
1/1 [==============================] - 0s 45ms/step
17
1/1 [==============================] - 0s 53ms/step
17
1/1 [==============================] - 0s 44ms/step
17
1/1 [==============================] - 0s 55ms/step
17
1/1 [==============================] - 0s 45ms/step
17
1/1 [==============================] - 0s 43ms/step
17
1/1 [==============================] - 0s 49ms/step
17
1/1 [==============================] - 0s 40ms/step
17
1/1 [==============================] - 0s 46ms/step
17
1/1 [==============================] - 0s 45ms/step
17
1/1 [==============================] - 0s 55ms/step
17
1/1 [==============================] - 0s 47ms/step
17
1/1 [==============================] - 0s 47ms/step
17
1/1 [==============================] - 0s 48ms/step
17
1/1 [==============================] - 0s 46ms/step
17
1/1 [==============================] - 0s 49ms/step
17
1/1 [====